In [4]:
import pandas as pd
import psycopg2

In [ ]:
import os

DB_NAME = os.getenv("DB_NAME", "PDATA")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASS = os.getenv("DB_PASSWORD", "postgres")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_POST = os.getenv("DB_PORT", "5432")

In [ ]:
def save_to_postgres(df, table_name='stock_data'):
    db_config = {
        'dbname': DB_NAME,
        'user': DB_USER,
        'password': DB_PASS,
        'host': DB_HOST,
        'port': DB_POST
    }
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    for _, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} (date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s);
        """
        cursor.execute(insert_query, (row['date'], row['open'], row['high'], row['low'], row['close'], row['volume']))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data saved to PostgreSQL table {table_name}")

In [ ]:

def read_from_postgres(table_name='stock_data'):
    db_config = {
        'dbname': DB_NAME,
        'user': DB_USER,
        'password': DB_PASS,
        'host': DB_HOST,
        'port': DB_POST
    }
    conn = psycopg2.connect(**db_config)
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Data read from PostgreSQL table {table_name}")
    return df